"SELECT TOP 1000000 gaia_source.source_id,gaia_source.ra,gaia_source.ra_error, \
                               gaia_source.dec,gaia_source.dec_error,gaia_source.parallax,gaia_source.parallax_error, \
                               gaia_source.parallax_over_error,gaia_source.phot_g_mean_mag,gaia_source.bp_rp, \
                               gaia_source.radial_velocity,gaia_source.radial_velocity_error, \
                               gaia_source.phot_variable_flag,gaia_source.teff_val,gaia_source.a_g_val \
            FROM gaiadr2.gaia_source \
            WHERE (gaiadr2.gaia_source.ra>=10.56014098062107 AND gaiadr2.gaia_source.ra<=334.56014098062107 \
               AND gaiadr2.gaia_source.dec>=-75.587831532659646 AND gaiadr2.gaia_source.dec<=89.587831532659646 \
               AND gaiadr2.gaia_source.parallax>=11.468624923468156 AND gaiadr2.gaia_source.parallax<=44.068624923468156 \
               AND gaiadr2.gaia_source.parallax_over_error>=1)"
Simbad, Ned, Vizier

In [3]:
import matplotlib.pyplot as plot
import numpy as np
from astropy.io.votable import parse_single_table
import astropy.units as u
import astropy.constants as c
from astropy.table import QTable
import astropy.coordinates as coord
from matplotlib import colors
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from PyAstronomy import pyasl
import warnings
warnings.filterwarnings('ignore')
from astroquery.gaia import Gaia
import utilities as util

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [4]:
#Querying methods

def read_gaia_vot(filename):
    return QTable(parse_single_table(filename).to_table())

def get_table_gaia_query(myquery):
    return QTable(Gaia.launch_job_async(myquery, dump_to_file = True).get_results())

In [5]:
#Astronomy methods

def luminosityFromMag(magnitude):
    return ((c.L_bol0).value * 10 ** (-0.4 * (magnitude).value))*u.erg

def distance(parallax):
    parallax_arcsec = parallax.to('arcsec').value
    return 1/parallax_arcsec * u.pc

def Magnitude_absolute_from_apparent(mag,dist):
    M = mag - 5*np.log10(dist.to('pc')/u.pc)*u.mag + 5.*u.mag
    return M

def angle_of_star(x,y,z):
    l=np.arctan2(y,x)
    b=np.arctan(z/np.sqrt(x**2+y**2))
    return l,b

In [6]:
a="trappist-1e"


In [7]:
ra, dec, pmra, pmdec = util.exoplanetProperMotion(a)
raminor = np.maximum(ra-350, 1)
ramajor = np.minimum(ra+350, 359)
decminor = np.maximum(dec-170, -89)
decmajor = np.minimum(dec+170, 89)

In [8]:
myquery1 = "SELECT TOP 500 *\
            FROM gaiadr2.gaia_source \
            WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',COORD1(EPOCH_PROP_POS("+ str(ra) +","+ str(dec) +",0,"+ str(pmra) +","+ str(pmdec) +",0,2000,2015.5)),COORD2(EPOCH_PROP_POS("+ str(ra) +","+ str(dec) +",0,"+ str(pmra) +","+ str(pmdec) +",0,2000,2015.5)),0.001388888888888889))=1"

b = get_table_gaia_query(myquery1)

for row in b:
    plx=row['parallax']
print(plx)
plxmajor=(np.minimum((plx.value)+20, 89))
plxminor=(np.maximum((plx.value)-70, 1))

80.45124261077767 mas


In [9]:
#Getting the stars between 1.4 and 2 parsecs away and ra between 281 and 310, and dec between 35 and 50

myquery2 = "SELECT TOP 1000000 gaia_source.source_id,gaia_source.ra,gaia_source.ra_error, \
                               gaia_source.dec,gaia_source.dec_error,gaia_source.parallax,gaia_source.parallax_error, \
                               gaia_source.parallax_over_error,gaia_source.phot_g_mean_mag,gaia_source.bp_rp, \
                               gaia_source.radial_velocity,gaia_source.radial_velocity_error, \
                               gaia_source.phot_variable_flag,gaia_source.teff_val,gaia_source.a_g_val, gaia_source.astrometric_pseudo_colour \
            FROM gaiadr2.gaia_source \
            WHERE (gaiadr2.gaia_source.ra>="+ str(raminor) + " AND gaiadr2.gaia_source.ra<="+ str(ramajor) +"\
               AND gaiadr2.gaia_source.dec>="+ str(decminor) +" AND gaiadr2.gaia_source.dec<="+ str(decmajor) +" \
               AND gaiadr2.gaia_source.parallax>="+ str(plxminor) +" AND gaiadr2.gaia_source.parallax<="+ str(plxmajor) +" \
               AND gaiadr2.gaia_source.parallax_over_error>=1)"

p = get_table_gaia_query(myquery2)

p

source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,phot_g_mean_mag,bp_rp,radial_velocity,radial_velocity_error,phot_variable_flag,teff_val,a_g_val,astrometric_pseudo_colour
,deg,mas,deg,mas,mas,mas,,mag,mag,km / s,km / s,,K,mag,1 / um
int64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,object,float32,float32,float64
3952456646420814208,185.5363048485253,0.10567704064629366,21.679595792117105,0.08749325932172013,10.451243541945608,0.15876465858072134,65.82853,16.990694046020508,3.1295223236083984,nan,nan,NOT_AVAILABLE,3693.2548828125,nan,1.23627781777384
2364357235633321856,8.52910676547077,0.14216349289481725,-17.450022231199902,0.1186865566841158,10.45124656033651,0.18620830924780674,56.12664,18.00710105895996,3.2301483154296875,nan,nan,NOT_AVAILABLE,nan,nan,1.3056534072554848
4093238252695846016,278.572383861155,0.6948452434314757,-19.027282184172662,0.6829755149509712,10.451246805280254,0.7125618079998939,14.667144,18.85920524597168,1.5197391510009766,nan,nan,NOT_AVAILABLE,nan,nan,1.6275073166227887
5872882519003780352,227.4937706996403,0.7315287138649058,-64.92854429702076,1.4090716068164635,10.45124880334386,2.2884003453466137,4.5670543,20.495744705200195,1.7268257141113281,nan,nan,NOT_AVAILABLE,nan,nan,1.6651915202708003
4090827504786855680,274.38045222944095,0.8825309835320339,-21.72980420267364,0.8054537048841168,10.451249642696167,0.9379138990998355,11.14308,19.353837966918945,nan,nan,nan,NOT_AVAILABLE,nan,nan,0.4263443396397708
5249408672412795008,145.75006302711517,0.058997336760760036,-64.37459105887312,0.06511916509290941,10.451249794861903,0.06654346813561647,157.05899,16.032583236694336,2.911214828491211,nan,nan,NOT_AVAILABLE,4618.93994140625,0.2460000067949295,1.2504423601023043
5854127698187999232,215.62284617604405,0.379155820501989,-63.12515094203675,0.5489943312954971,10.451254476370963,0.8780804700466502,11.902388,18.180984497070312,2.0004348754882812,nan,nan,NOT_AVAILABLE,nan,nan,1.3871361606291017
5903211001080727424,226.8910342330614,1.054896854226366,-48.54173416308973,0.8333569298578672,10.451254809749084,1.3605554313234236,7.681609,20.56606674194336,1.1787090301513672,nan,nan,NOT_AVAILABLE,nan,nan,1.1973099189830094


In [10]:
largedistance = distance(p['parallax'])
K2Sky=coord.SkyCoord(ra=p['ra'],dec=p['dec'],distance=largedistance)
K2GC=K2Sky.galactocentric
K2Sky

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(185.53630485,  21.67959579, 95.68239377),
     (  8.52910677, -17.45002223, 95.68236614),
     (278.57238386, -19.02728218, 95.6823639 ), ...,
     (272.90682493, -26.02707381, 11.24959389),
     ( 67.25020492,  21.92345105, 11.24073242),
     (269.85496935, -30.40741295, 11.23695294)]>

In [11]:
#Finding Sky Coordinates of Kepler 452
ra= ra*u.degree
dec= dec*u.degree
distance=distance(plx)
K218b=coord.SkyCoord(ra,dec,distance)
K218b

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (346.62233333, -5.04144444, 12.42988881)>

In [12]:
#Converting Kepler 4452 Sky Coordinates into Galactocentric Coordinates
K218bG=K218b.galactocentric
K218bG

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.3 kpc, galcen_v_sun=(11.1, 232.24, 7.25) km / s, z_sun=27.0 pc, roll=0.0 deg): (x, y, z) in pc
    (-8297.62019463, 6.41038235, 16.60996854)>

In [13]:
#Finding the distances between the x,y,z of the stars in our query and Kepler 452's x,y,z
x=(K2GC.x-K218bG.x)
y=(K2GC.y-K218bG.y)
z=(K2GC.z-K218bG.z)

print(x)
print(y)
print(z)

[ -6.78126225  -5.53726055  90.11369429 ...   8.83998571 -12.99856494
   8.87980721] pc
[-19.91985378  10.73787766  16.73751652 ...  -5.35352321  -5.56057111
  -6.36003771] pc
[105.00955017 -83.6886781    1.88332946 ...   9.65775923   6.93471082
   9.70103073] pc


In [14]:
#Using our method angle_of_star to solve for L,B in Galactic
L,B=angle_of_star(x,y,z)

In [15]:
#Finding the apparent magnitudes of the stars from the neighboring stars
p['Mg'] = Magnitude_absolute_from_apparent(p['phot_g_mean_mag'], largedistance)
p['otherDistance'] = np.sqrt(x.value**2+y.value**2+z.value**2)*u.pc
p['m'] = (p['Mg'].value - 5 + 5*np.log10(p['otherDistance'].value))*u.mag
p['wavelength']=((1/p['astrometric_pseudo_colour'])*1000)*u.nm/u.um
p['wavelength']

<Quantity [808.87967544, 765.90004242, 614.43656184, ...,          nan,
           691.45668447, 587.05468224] nm>

In [18]:
fig = plot.figure(figsize=(12.8, 9.6))
ax = fig.add_subplot(1,1,1)


plot.scatter((L.to('degree')).value, (B.to('degree').value), s = 1, c = (p['wavelength']), alpha = 0.1, cmap = "nipy_spectral")
ax.set_facecolor("Black")
plot.clim(400, 800)

plot.scatter((L.to('degree')).value, (B.to('degree').value), s = 1, c = (p['m']), alpha = 0.1, cmap = "gray")
ax.set_facecolor("Black")

plot.xlabel("Right Ascension (Degrees)")
plot.ylabel("Declination (Degrees)")

plot.xlim(180,-180)
plot.ylim(-90,90)
plot.show()

In [19]:
len(p)

603290